In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import re 
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

train = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", delimiter = "\t")
test = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip", delimiter = "\t")

train.shape
test.shape

In [ ]:
def clean_data(data): 
    num_reviews = data["review"].size
    data_reviews_clean = []
    for i in range(0, num_reviews): 
        review_text = BeautifulSoup(data["review"][i]).get_text() 
        letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
        words = letters_only.lower().split()        
        stopwordss = set(stopwords.words("english"))                  
        meaningful_words = [w for w in words if not w in stopwordss]  

        data_reviews_clean.append(" ".join(meaningful_words))
        
    return data_reviews_clean;

In [ ]:
train_clean = clean_data(train)
vector_train = CountVectorizer(analyzer = "word",tokenizer = None, preprocessor = None,stop_words = None, max_features = 10000)
train_features = vector_train.fit_transform(train_clean)   
train_features = train_features.toarray()

forest_clf = RandomForestClassifier(n_estimators = 150)  
forest_clf.fit(train_features,train["sentiment"])

test_clean = clean_data(test)

vector_test = vector_train.transform(test_clean)
test_features = vector_test.toarray()

result = forest_clf.predict(test_features)
result

In [ ]:
submission = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
submission

In [ ]:

submission.to_csv( "submission.csv", index=False, quoting=3 )